In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gradio

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import gradio as gr
import numpy as np
from PIL import Image
import os  # Pastikan untuk mengimpor modul os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 117.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.2 MB/s eta 0:00:00


In [ ]:
train_dir = '/content/drive/MyDrive/vechile_dataset/train'
validation_dir = '/content/drive/MyDrive/vechile_dataset/validation'
test_dir = '/content/drive/MyDrive/vechile_dataset/test'

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

Found 884 images belonging to 2 classes.
Found 884 images belonging to 2 classes.


In [ ]:
def count_images(directory):
    count = 0
    for root, dirs, files in os.walk(directory):
        count += len([file for file in files if file.endswith('.jpg') or file.endswith('.png')])
    return count

num_train_images = count_images(train_dir)
num_validation_images = count_images(validation_dir)
num_test_images = count_images(test_dir)

print(f"Jumlah gambar di train set: {num_train_images}")
print(f"Jumlah gambar di validation set: {num_validation_images}")
print(f"Jumlah gambar di test set: {num_test_images}")

# Menghitung steps_per_epoch dan validation_steps
batch_size = 32  # Ukuran batch yang digunakan pada generator
steps_per_epoch = num_train_images // batch_size
validation_steps = num_validation_images // batch_size

Jumlah gambar di train set: 884
Jumlah gambar di validation set: 884
Jumlah gambar di test set: 474


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,  # Jumlah epoch bisa disesuaikan sesuai kebutuhan
    validation_data=validation_generator,
    validation_steps=validation_steps
)

# Opsional: Cetak riwayat pelatihan untuk melihat akurasi dan loss per epoch
print(history.history)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 293s 11s/step - accuracy: 0.6460 - loss: 0.7979 - val_accuracy: 0.9329 - val_loss: 0.2187
Epoch 2/10
 1/27 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.9688 - loss: 0.2461

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


27/27 ━━━━━━━━━━━━━━━━━━━━ 20s 697ms/step - accuracy: 0.9688 - loss: 0.2461 - val_accuracy: 0.9468 - val_loss: 0.1486
Epoch 3/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 78s 3s/step - accuracy: 0.9650 - loss: 0.0982 - val_accuracy: 0.9433 - val_loss: 0.1319
Epoch 4/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 20s 703ms/step - accuracy: 0.8438 - loss: 0.2612 - val_accuracy: 0.9838 - val_loss: 0.0639
Epoch 5/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 142s 3s/step - accuracy: 0.9411 - loss: 0.1283 - val_accuracy: 0.9803 - val_loss: 0.0729
Epoch 6/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 21s 763ms/step - accuracy: 0.9062 - loss: 0.1465 - val_accuracy: 0.9815 - val_loss: 0.0565
Epoch 7/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.9874 - loss: 0.0512 - val_accuracy: 0.9965 - val_loss: 0.0153
Epoch 8/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 20s 670ms/step - accuracy: 0.9688 - loss: 0.0708 - val_accuracy: 0.9896 - val_loss: 0.0315
Epoch 9/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 108s 3s/step - accuracy: 0.9858 - loss: 0.0473 - val_accuracy: 0.9988 - val_lo

In [ ]:
def classify_image(img):
    img = img.resize((150, 150))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)
    return 'Car' if prediction < 0.5 else 'Motorcycle'

interface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type='pil'),
    outputs="text",
    title="Vehicle Classifier",
    description="Upload a vehicle image (car or motorcycle) to classify it using a CNN.",
    allow_flagging="never"  # Nonaktifkan tombol flag
)

interface.launch()


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cfdee225608543a973.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
